In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import time

In [13]:
data1 = pd.read_csv("new_only_A_merge.tsv",sep=" ",names=["chr","pos","flag","ipdratio","6ma_flag","context"])
data1.set_index(["chr","pos","flag"],inplace=True) 
data1

ipdratio  6ma_flag                context
chr    pos    flag                                           
ecoli1 12     1        0.541         0  AAGGCATATAAACCTATTATT
       13     1        0.492         0  CAAGGCATATAAACCTATTAT
       14     0        0.449         0  TAATAGGTTTATATGCCTTGT
       15     1        0.689         0  GACAAGGCATATAAACCTATT
       16     0        0.499         0  ATAGGTTTATATGCCTTGTCA
...                      ...       ...                    ...
ecoli2 121774 0        0.711         0  CCAGCGTGGAAAAGGCTGGGT
       121775 0        0.645         0  CAGCGTGGAAAAGGCTGGGTG
       121776 0        0.483         0  AGCGTGGAAAAGGCTGGGTGG
       121780 1        0.498         0  AATGCCACCCAGCCTTTTCCA
       121784 1        0.760         0  TTGAAATGCCACCCAGCCTTT

[2424431 rows x 3 columns]

In [10]:
data = pd.read_csv("test.tsv",sep=" ",names=["chr","pos","flag","ipd","pw"])
g=open("./comput_result","w")
data.set_index(["chr","pos","flag"],inplace=True)
data
for i in data.index.levels[0]:
    data1=data.loc[i]
    for j in set([p[0] for p in data1.index]):
            data2=data1.loc[j]
            for f in set(data2.index):
                if len(data2.loc[f]) >=5:
                    tensoripd=np.array(data2.loc[f].ipd)
                    tensorpw=np.array(data2.loc[f].pw)
                    ipd="-".join(str(n) for n in list(tensoripd))
                    pw="-".join(str(n) for n in list(tensorpw))
                    print(i,j,f,ipd,pw,file=g)
                else:
                    continue
g.close()

In [11]:
data2=pd.read_csv("comput_result",sep=" ",names=["chr","pos","flag","ipd_list","pw_list"])
data2.set_index(["chr","pos","flag"],inplace=True) 
data2

ipd_list       pw_list
chr    pos  flag                              
ecoli1 1    0         9-2-5-0-10    6-21-7-4-6
       2    0        58-20-7-2-6   34-6-9-3-26
       3    0         18-0-7-4-0     3-6-4-4-5
       5    0       50-14-17-4-9    8-23-3-2-3
       6    0     211-11-9-25-19    3-3-1-6-13
...                          ...           ...
       5055 0        8-18-4-11-1  10-8-23-7-11
       5056 0        2-5-3-12-13     6-3-5-6-2
       5059 0        4-8-4-15-19    14-8-4-8-2
       5064 0        4-10-38-0-5    5-1-4-3-23
       5065 0        3-26-60-6-0    6-4-4-5-10

[916 rows x 2 columns]

In [14]:
df_final=pd.concat([data2,data1],axis=1, join='inner')
df_final=df_final.reset_index()
df_final.set_index(["chr","pos","flag","6ma_flag"],inplace=True)

df_final.to_csv("./final.csv")
df_final

ipd_list       pw_list  ipdratio  \
chr    pos  flag 6ma_flag                                          
ecoli1 14   0    0         20-18-40-6-19    6-4-3-15-7     0.449   
       16   0    0           9-4-41-29-2     8-6-6-8-6     0.499   
       28   0    0         14-22-22-1-17     7-8-8-7-4     0.572   
       32   0    0           3-25-6-19-3   13-12-4-7-8     0.423   
       33   0    0          99-27-2-6-39    4-13-1-5-4     0.591   
...                                  ...           ...       ...   
       5055 0    0           8-18-4-11-1  10-8-23-7-11     1.082   
       5056 0    0           2-5-3-12-13     6-3-5-6-2     0.938   
       5059 0    1           4-8-4-15-19    14-8-4-8-2     1.428   
       5064 0    0           4-10-38-0-5    5-1-4-3-23     0.966   
       5065 0    0           3-26-60-6-0    6-4-4-5-10     1.100   

                                         context  
chr    pos  flag 6ma_flag                         
ecoli1 14   0    0         TAATAGGTTTATATGCCTTGT  
       16   0    0         ATAGGTTTATATGCCTTGTCA  
       28   0    0         GCCTTGTCACATCTAACCATT  
       32   0    0         TGTCACATCTAACCATTCATG  
       33   0    0         GTCACATCTAACCATTCATGT  
...                                          ...  
       5055 0    0         TCTGACCGTGAAGCAGGAAAA  
       5056 0    0         CTGACCGTGAAGCAGGAAAAA  
       5059 0    1         ACCGTGAAGCAGGAAAAAAGT  
       5064 0    0         GAAGCAGGAAAAAAGTTTACG  
       5065 0    0         AAGCAGGAAAAAAGTTTACGC  

[910 rows x 4 columns]

In [4]:
def get_DNA_seq_tensor(seq):
    seq=seq.upper()
    df=np.zeros((4,len(seq)),dtype="int")
    df = pd.DataFrame(df,index=["A","T","C","G"])
    for i in range(len(seq)):
        df.loc[seq[i],i]=1 
    return np.array(df).reshape(1,-1)[0]

In [5]:
def re_code(piece):
    col1=get_cluster([int(i) for i in piece["ipd_list"].split("-")],5)
    col2=get_cluster([int(i) for i in piece["pw_list"].split("-")],5)
    col3=list(get_DNA_seq_tensor(piece["context"]))
    return col1+col2+col3

In [6]:
def get_cluster(tensor,n):
    result1=[]
    result2=[]
    if len(set(tensor)) <n:
        tensor.sort()
        result1=tensor[int((len(tensor)-5)/2):int((len(tensor)-5)/2)+5]
        result2=[0,0,0,0,0]
    else:
        y = np.array(tensor).reshape(-1, 1)
        km = KMeans(n_clusters=n,random_state=1)
        km.fit(y)
        for i in range(n):
            result1.append(np.mean(y[km.labels_==i]))
            result2.append(np.std(y[km.labels_==i]))
    result1.sort()
    result2.sort()
    return result1+result2

In [8]:
df_final=pd.read_csv("./final.csv",sep=",",names=["chr","pos","flag","6ma_flag","ipd_list","pw_list","ipdratio","context"],header=1)
colname1=["clu_ipd"+str(i) for i in range(10)]
colname2=["clu_pw"+str(i) for i in range(10)]
colname3=["code"+str(i) for i in range(84)]
df_final = pd.concat([df_final, pd.DataFrame(columns=colname1+colname2+colname3)])
temp=df_final.apply(re_code,axis=1)
df_final[colname1+colname2+colname3]=pd.DataFrame(list(map(np.array,temp)))
df_final["final_tag"]=df_final["6ma_flag"]
df_final["final_tag"]=df_final["final_tag"].astype(int)
df_final=df_final.drop(['chr', 'pos', 'flag', '6ma_flag', 'context','ipd_list', 'pw_list',],axis=1)
df_final=df_final.round(3)
# df_final.to_csv("./final_code.csv",header=False,index=False)
df_final

,ipdratio,clu_ipd0,clu_ipd1,clu_ipd2,clu_ipd3,clu_ipd4,clu_ipd5,clu_ipd6,clu_ipd7,clu_ipd8,...,code75,code76,code77,code78,code79,code80,code81,code82,code83,final_tag
0,0.499,2.0,4.0,9.0,29.0,41.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,0.572,1.0,14.0,17.0,22.0,22.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.423,3.0,3.0,6.0,19.0,25.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.591,2.0,6.0,27.0,39.0,99.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,0.536,1.0,3.0,5.0,8.0,33.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,1.082,1.0,4.0,8.0,11.0,18.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
905,0.938,2.0,3.0,5.0,12.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0
906,1.428,4.0,4.0,8.0,15.0,19.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
907,0.966,0.0,4.0,5.0,10.0,38.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0
